In [207]:
#Instalando mesa con la librerías que se van a ocupar
!pip install numpy scipy matplotlib seaborn scikit-learn mesa==3.0 -q

In [208]:
# ------------------------------
# Valores para el mapa
# ------------------------------
EMPTY = 0
WALL = 1
VICTIM = 4
FALSE_ALARM = 5


# ------------------------------
# Diccionario de direcciones (fila, col, lado)
# ------------------------------
DIRECTIONS = {
    "up":    (-1, 0, 0),  # fila-1, col, lado 0
    "left":  (0, -1, 1),  # fila, col-1, lado 1
    "down":  (1, 0, 2),   # fila+1, col, lado 2
    "right": (0, 1, 3)    # fila, col+1, lado 3
}

In [209]:
#Conexion
import socket
import json
#Conexión
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import  random
from os import read
from heapq import heappush, heappop


In [210]:
fileref = open("ArchProj.txt", "r")
texto = fileref.readlines()
fileref.close()
grids = texto[:6]
victim = texto[6:15]
Fire = texto[15:24]
Door_lines = texto[24:37]
Access_point = texto[37:43]
print(grids)
print(victim)
print(Fire)
print(Door_lines)
print(Access_point)


# -------------------------------
# Leer txt y colocar en variables
# -------------------------------
fileref = open("ArchProj.txt", "r")
texto = fileref.readlines()
fileref.close()
grids = texto[:6]
victim = texto[6:15]
Fire = texto[15:24]
Door_lines = texto[24:37]
Access_point = texto[37:43]

# ------------------------------
# Parsear listas desde el archivo
# funciones realizada con ayuda de IA
# ------------------------------
def parse_positions(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 2:
            x, y = map(int, parts[:2])
            coords.append((y-1, x-1))  #(row, col)
    return coords

def parse_positions_v_f(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 3:
            y, x = map(int, parts[:2])  # OJO: primero fila (y), luego columna (x)
            kind = parts[2]
            coords.append((y-1, x-1, kind))
    return coords

def parse_doors(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 4:
            x1, y1, x2, y2 = map(int, parts[:4])
            coords.append((y1-1, x1-1, y2-1, x2-1))  # (row,col,row,col)
    return coords


fire_coords = parse_positions(Fire)
door_coords = parse_doors(Door_lines)
print("Puertas:", door_coords)
access_coords = parse_positions(Access_point)
print("Puntos de acceso:", access_coords)
false_alarm_victims_coords = parse_positions_v_f(victim)
print("Falsas alarmas y víctimas:", false_alarm_victims_coords)


['1011 1110 1011 1010 1000 1100 1001 1100\n', '1000 1000 1010 1110 0011 0110 0011 0110\n', '0011 0110 1101 1101 1111 1011 1110 1101\n', '1011 1010 0110 0111 1011 1010 1010 0110\n', '1001 1110 1001 1100 1101 1101 1101 1101\n', '0111 1011 0010 0110 0111 0111 0111 0111\n']
['1 1 f\n', '1 2 f\n', '1 8 v\n', '2 4 v\n', '3 5 v\n', '4 1 v\n', '6 2 v\n', '6 5 v\n', '6 7 v\n']
['1 3\n', '1 7\n', '2 3\n', '3 3\n', '5 2\n', '5 5\n', '5 7\n', '5 8\n', '6 3\n']
['1 2 1 3\n', '1 6 1 7\n', '2 4 2 5\n', '2 8 3 8\n', '3 3 3 4\n', '3 4 3 5\n', '3 5 3 6\n', '3 6 4 6\n', '4 4 5 4\n', '4 5 5 5\n', '4 6 5 6\n', '5 2 5 3\n', '6 6 6 7\n']
['1 4\n', '1 6\n', '2 1\n', '4 6\n', '6 4\n', '6 6\n']
Puertas: [(1, 0, 2, 0), (5, 0, 6, 0), (3, 1, 4, 1), (7, 1, 7, 2), (2, 2, 3, 2), (3, 2, 4, 2), (4, 2, 5, 2), (5, 2, 5, 3), (3, 3, 3, 4), (4, 3, 4, 4), (5, 3, 5, 4), (1, 4, 2, 4), (5, 5, 6, 5)]
Puntos de acceso: [(3, 0), (5, 0), (0, 1), (5, 3), (3, 5), (5, 5)]
Falsas alarmas y víctimas: [(0, 0, 'f'), (0, 1, 'f'), (0, 7, 'v

In [211]:
###############################################################################

# ------------------------------
# Clase pared
# ------------------------------
class Cell:
    def __init__(self, up, left, down, right):
        self.positions = [up, left, down, right]
        self.damages = [2,2,2,2]

    def get_damages(self, side = None):
        if side == None:
            return self.damages
        else:
            return self.damages[side]

    def get_positions(self, side = None):
        if side != None:
            return self.positions[side]
        else:
            return self.positions

    def set_damage(self, side, damage):
        if self.damages[side] <= 0:
            return

        self.damages[side] -= damage
        if self.damages[side] <= 0:
            print(f"Pared destruida en lado {side}")

    def is_empty(self):
      for position in self.positions:
        if position != EMPTY:
          return False
        else:
          return True

    def set_cell(self,values):
        if len(values) == 4:
            self.positions = values
        return self.positions

# ------------------------------
# Crear matriz de celdas 6x8 a
# partir de `grids`
# Función creada con ayuda de IA
# ------------------------------
height, width = 6, 8
matriz_pared = [[None for _ in range(width)] for _ in range(height)]

for i in range(height): # recorre las 6 fila
  fila = grids[i].strip().split() # separa en bloques (ej: '1011', '1110')
  for j in range(width): # recorre las 8 columnas
    valores = [int(x) for x in fila[j]] # convierte '1011' → [1,0,1,1]
    matriz_pared[i][j] = Cell(*valores) # desempaqueta en up,left,down,righ

for i in range(height):
  for j in range(width):
    print(matriz_pared[i][j].get_positions(), end=" ")
  print()

  for x,y in access_coords:
    matriz_pared[y][x].set_cell([0,0,0,0]) # acceso libre

###############################################################################
# ------------------------------
# Clase Puertas
# ------------------------------
# 0 cerrada
# 1 abierta
# 2 destruida
class Door:
    CLOSED = 0
    OPEN = 1
    DESTROYED = 2

    def __init__(self, r1, c1, r2, c2):
        self.state = Door.CLOSED
        self.cell1 = (r1, c1)
        self.cell2 = (r2, c2)

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

    def open(self):
        if self.state == Door.CLOSED:
            self.state = Door.OPEN

    def close(self):
        if self.state == Door.OPEN:
            self.state = Door.CLOSED

    def destroy(self):
        self.state = Door.DESTROYED

    def connects(self, cell):
        if cell == self.cell1:
            return self.cell2
        elif cell == self.cell2:
            return self.cell1
        return None

    def blocks_passage(self):
        return self.state == Door.CLOSED


###############################################################################
door_list = []
for (r1, c1, r2, c2) in door_coords:
    door = Door(r1, c1, r2, c2)
    door_list.append(door)

cell_doors = {}

for door in door_list:
    for cell in [door.cell1, door.cell2]:
        if cell not in cell_doors:
            cell_doors[cell] = []
        cell_doors[cell].append(door)

###############################################################################
#----------------------
# clase fire
#----------------------
class Fire:
    EMPTY = 0
    SMOKE = 1
    FIRE = 2
    EXPLODED = 3

    def __init__(self, row, col):
        self.state = Fire.EMPTY
        self.row = row
        self.col = col
        self.to_explode = False

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

    def get_position(self):
        return (self.row, self.col)

    def get_neighbors(self, matriz_pared):
        neighbors = []
        # arriba, derecha, abajo, izquierda
        directions = [(-1,0,0), (0,1,1), (1,0,2), (0,-1,3)]
        for dr, dc, side in directions:
            nr, nc = self.row + dr, self.col + dc
            if 0 <= nr < len(matriz_pared) and 0 <= nc < len(matriz_pared[0]):
                if matriz_pared[self.row][self.col].get_positions(side) == 0 and \
                   matriz_pared[nr][nc].get_positions((side + 2) % 4) == 0:
                    neighbors.append((nr, nc, side))
        return neighbors

    def advance_fire(self):
        r = random.randint(0, len(self.fire_grid)-1)
        c = random.randint(0, len(self.fire_grid[0])-1)
        target = self.fire_grid[r][c]

        if target.state == Fire.EMPTY:
            target.state = Fire.SMOKE
        elif target.state == Fire.SMOKE:
            target.state = Fire.FIRE
        elif target.state == Fire.FIRE:
            target.to_explode = True


    def propagate_explosion(self, matriz_fire, matriz_pared, model=None):
        # Explosión en 4 direcciones
        for nr, nc, side in self.get_neighbors(matriz_pared):
            vecino = matriz_fire[nr][nc]
            if vecino.state == Fire.EMPTY:
                vecino.state = Fire.SMOKE
            elif vecino.state == Fire.SMOKE:
                vecino.state = Fire.FIRE

            # Daño a paredes
            if matriz_pared[nr][nc].get_positions(side) == 1:
                matriz_pared[nr][nc].set_damage(side, 1)
                model.total_damage += 1
                print(f"Daño en pared por fuego en: {(nr, nc)} lado {side}")

        self.to_explode = False

    def set_smoke(self, matriz_fire, matriz_pared):
        # Paso de flashover: humo adyacente a fuego
        for nr, nc, _ in self.get_neighbors(matriz_pared):
            if matriz_fire[nr][nc].state == Fire.SMOKE:
                matriz_fire[nr][nc].state = Fire.FIRE

###############################################################################
print(fire_coords)
height, width = 6, 8
matriz_fire = [[Fire(i,j) for j in range(width)] for i in range(height)]

for (x,y) in fire_coords:
  matriz_fire[y][x].set_state(2)

for i in range(height):
    for j in range(width):
        print(matriz_fire[i][j].get_state(), end=" ")
    print()

###############################################################################
# Clase Victim or False Alarm
###############################################################################
class POI:
    VICTIM = "v"
    FALSE = "f"

    def __init__(self, row, col, role):
        self.row = row
        self.col = col
        self.role = role
        self.revealed = False

    def reveal(self):
        self.revealed = True

    def is_victim(self):
        return self.role == POI.VICTIM and self.revealed

    def is_false(self):
        return self.role == POI.FALSE and self.revealed

    def get_position(self):
        return (self.row, self.col)


###############################################################################
# Corrección de matriz de POIs
###############################################################################

matriz_POI = [[None for j in range(width)] for i in range(height)]
for (row, col, role) in false_alarm_victims_coords:   # fila, col, rol
        matriz_POI[row][col] = POI(row, col, role)

"""
print("Matriz de POIs:")
for i in range(height):
    for j in range(width):
        if matriz_POI[i][j] is None:
            print(" . ", end=" ")
        else:
            print(f" {matriz_POI[i][j].role} ", end=" ")
    print()
"""


[1, 0, 1, 1] [1, 1, 1, 0] [1, 0, 1, 1] [1, 0, 1, 0] [1, 0, 0, 0] [1, 1, 0, 0] [1, 0, 0, 1] [1, 1, 0, 0] 
[0, 0, 0, 0] [1, 0, 0, 0] [1, 0, 1, 0] [1, 1, 1, 0] [0, 0, 1, 1] [0, 1, 1, 0] [0, 0, 1, 1] [0, 1, 1, 0] 
[0, 0, 1, 1] [0, 1, 1, 0] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 1, 1] [1, 0, 1, 1] [1, 1, 1, 0] [1, 1, 0, 1] 
[1, 0, 1, 1] [1, 0, 1, 0] [0, 1, 1, 0] [0, 1, 1, 1] [1, 0, 1, 1] [0, 0, 0, 0] [1, 0, 1, 0] [0, 1, 1, 0] 
[1, 0, 0, 1] [1, 1, 1, 0] [1, 0, 0, 1] [1, 1, 0, 0] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 0, 1] 
[0, 1, 1, 1] [1, 0, 1, 1] [0, 0, 1, 0] [0, 0, 0, 0] [0, 1, 1, 1] [0, 0, 0, 0] [0, 1, 1, 1] [0, 1, 1, 1] 
[(2, 0), (6, 0), (2, 1), (2, 2), (1, 4), (4, 4), (6, 4), (7, 4), (2, 5)]
0 0 2 0 0 0 2 0 
0 0 2 0 0 0 0 0 
0 0 2 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 2 0 0 2 0 2 2 
0 0 2 0 0 0 0 0 


'\nprint("Matriz de POIs:")\nfor i in range(height):\n    for j in range(width):\n        if matriz_POI[i][j] is None:\n            print(" . ", end=" ")\n        else:\n            print(f" {matriz_POI[i][j].role} ", end=" ")\n    print()\n'

In [212]:

# ------------------------------
# Función A* para buscar camino
# ------------------------------
def a_star(start, goal, matriz_pared):
    open_set = []
    heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}

    def heuristic(a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])  # Manhattan

    def door_allows_passage(cell1, cell2):
        if cell1 in cell_doors:
            for door in cell_doors[cell1]:
                if door.connects(cell1) == cell2:
                    # Sólo se puede pasar si puerta está ABierta o DESTRUIDA
                    return door.state != Door.CLOSED
        return False

    while open_set:
        _, current = heappop(open_set)
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]

        row, col = current
        for drow, dcol, side in [(-1,0,0),(0,1,1),(1,0,2),(0,-1,3)]:
            nrow, ncol = row+drow, col+dcol
            if 0 <= nrow < len(matriz_pared) and 0 <= ncol < len(matriz_pared[0]):

                # No planear hacia fuego activo
                if matriz_fire[nrow][ncol].state == Fire.FIRE:
                    continue

                # Pared entre current y neighbor
                if matriz_pared[row][col].get_positions(side) == 1:
                    if not door_allows_passage((row, col), (nrow, ncol)):
                        continue

                # Pared del lado opuesto del vecino
                if matriz_pared[nrow][ncol].get_positions((side+2)%4) == 1:
                    if not door_allows_passage((nrow, ncol), (row, col)):
                        continue

                neighbor = (nrow, ncol)
                tentative = g_score[current] + 1
                if tentative < g_score.get(neighbor, float("inf")):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative
                    f_score = tentative + heuristic(neighbor, goal)
                    heappush(open_set, (f_score, neighbor))
    return []

def get_door_between(cell1, cell2):
    if cell1 in cell_doors:
        for door in cell_doors[cell1]:
            if door.connects(cell1) == cell2:
                return door
    return None

def nearest_reachable_closed_door(from_cell, matriz_pared):
    # Devuelve una celda adyacente a una puerta CERRADA a la que SÍ haya ruta
    height, width = len(matriz_pared), len(matriz_pared[0])
    candidates = []  # (dist_manhattan, (r_ady, c_ady), door)
    fr, fc = from_cell
    seen_pairs = set()

    for (cell, doors) in cell_doors.items():
        for door in doors:
            if door.state == Door.CLOSED:
                c1, c2 = door.cell1, door.cell2
                # intenta llegar al lado c1 o al lado c2
                for dest in (c1, c2):
                    if 0 <= dest[0] < height and 0 <= dest[1] < width:
                        pair_key = (door.cell1, door.cell2, dest)
                        if pair_key in seen_pairs:
                            continue
                        seen_pairs.add(pair_key)
                        route = a_star(from_cell, dest, matriz_pared)
                        if route:
                            dman = abs(fr - dest[0]) + abs(fc - dest[1])
                            candidates.append((dman, dest, door))
    if not candidates:
        return None, None
    candidates.sort(key=lambda x: x[0])
    _, dest_cell, door = candidates[0]
    return dest_cell, door


In [213]:

# ------------------------------
# Clase RanaAgent con estrategia A*
# ------------------------------
class RanaAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.Initial_AP = 4  # si quieres, súbelo a 6–8
        self.AP = self.Initial_AP
        self.carry = False
        self.route = []
        self.target = None
        self.intent = "poi"  # "poi", "exit" o "door"

    # --- costos de acciones ---
    MOVE_COST = 1
    OPEN_DOOR_COST = 1
    BREACH_DOOR_COST = 2
    EXTINGUISH_FIRE_COST = 2   # FUEGO -> HUMO
    CLEAR_SMOKE_COST = 1       # HUMO -> VACÍO

    def sum_AP(self):
        self.AP = min(self.AP + self.Initial_AP, 8)
        return self.AP

    # -------- puerta ----------
    def open_or_breach(self, door):
        if door.state != Door.CLOSED:
            return True
        if self.AP >= self.OPEN_DOOR_COST:
            door.open()
            self.AP -= self.OPEN_DOOR_COST
            print(" Rana abrió puerta")
            return True
        # si no alcanza para abrir, intenta destruir si hay AP suficiente
        if self.AP >= self.BREACH_DOOR_COST:
            door.destroy()
            self.AP -= self.BREACH_DOOR_COST
            print(" Rana destruyó puerta")
            return True
        return False

    # -------- fuego -----------
    def extinguish_cell(self, r, c):
        cell = matriz_fire[r][c]
        if cell.state == Fire.FIRE and self.AP >= self.EXTINGUISH_FIRE_COST:
            cell.state = Fire.SMOKE
            self.AP -= self.EXTINGUISH_FIRE_COST
            print(f" Rana bajó FUEGO a HUMO en {(r,c)}")
            return True
        if cell.state == Fire.SMOKE and self.AP >= self.CLEAR_SMOKE_COST:
            cell.state = Fire.EMPTY
            self.AP -= self.CLEAR_SMOKE_COST
            print(f" Rana limpió HUMO en {(r,c)}")
            return True
        return False

    # ---------- movimiento según ruta ----------
    def follow_route(self):
        while self.AP > 0 and self.route:
            nrow, ncol = self.route[0]
            current = (self.pos[1], self.pos[0])
            neighbor = (nrow, ncol)

            # 1) si hay fuego en la celda destino, intenta apagar
            if matriz_fire[nrow][ncol].state in (Fire.FIRE, Fire.SMOKE):
                if self.extinguish_cell(nrow, ncol):
                    # Recalcular ruta tras cambiar estado del fuego
                    if self.target:
                        self.route = a_star((self.pos[1], self.pos[0]), self.target, matriz_pared)
                        if self.route and self.route[0] == (self.pos[1], self.pos[0]):
                            self.route.pop(0)
                    return
                else:
                    # sin AP para apagar → salir del movimiento este turno
                    return

            # 2) si hay puerta cerrada, abrir/romper
            door = get_door_between(current, neighbor)
            if door and door.state == Door.CLOSED:
                if self.open_or_breach(door):
                    # Recalcular ruta después de habilitar el paso
                    if self.target:
                        self.route = a_star((self.pos[1], self.pos[0]), self.target, matriz_pared)
                        if self.route and self.route[0] == (self.pos[1], self.pos[0]):
                            self.route.pop(0)
                    return
                else:
                    return  # no hay AP para abrir/romper

            # 3) moverse
            if self.AP >= self.MOVE_COST:
                self.route.pop(0)
                self.model.grid.move_agent(self, (ncol, nrow))
                self.AP -= self.MOVE_COST
                print(f"Rana movida a {(nrow,ncol)}")
            else:
                return

    # ---------- planificación ----------
    def plan_to_nearest_poi(self, matriz_pared, matriz_POI):
        row, col = self.pos[1], self.pos[0]
        pois = []
        for r in range(len(matriz_POI)):
            for c in range(len(matriz_POI[0])):
                poi = matriz_POI[r][c]
                if poi and not poi.revealed:
                    pois.append((poi.row, poi.col))

        if not pois:
            print(" No hay POIs activos ahora mismo")
            self.target, self.route = None, []
            return False

        pois.sort(key=lambda p: abs(row - p[0]) + abs(col - p[1]))
        for poi_pos in pois:
            route = a_star((row, col), poi_pos, matriz_pared)
            if route:
                self.intent = "poi"
                self.target = poi_pos
                self.route = route
                print(f"Rana planifica hacia POI {poi_pos} con ruta {route}")
                return True

        # Ningún POI alcanzable → plan B: buscar puerta cerrada alcanzable para abrir región
        dest_cell, door = nearest_reachable_closed_door((row, col), matriz_pared)
        if dest_cell:
            route = a_star((row, col), dest_cell, matriz_pared)
            if route:
                self.intent = "door"
                self.target = dest_cell
                self.route = route
                print(f" Rana irá a puerta cerrada en {dest_cell} para abrir/romper")
                return True

        print(" Rana no encontró ruta hacia ningún POI ni puerta cerrada alcanzable")
        self.target, self.route = None, []
        return False

    def plan_to_exit(self, matriz_pared, access_coords):
        row, col = self.pos[1], self.pos[0]
        exits = [(r,c) for (r,c) in access_coords]
        exits.sort(key=lambda e: abs(row-e[0])+abs(col-e[1]))
        for exit in exits:
            route = a_star((row,col), exit, matriz_pared)
            if route:
                self.intent = "exit"
                self.target = exit
                self.route = route
                print(f"Rana planifica salida hacia {exit} con ruta {route}")
                return True
        print(" Rana no encontró salida accesible")
        self.target, self.route = None, []
        return False

    # ---------- ciclo de vida ----------
    def step(self):
        # Si el target actual ya no existe (POI muerto o retirado), replanificar
        if self.intent == "poi" and self.target:
            tr, tc = self.target
            if not (0 <= tr < len(self.model.poi_grid) and 0 <= tc < len(self.model.poi_grid[0])) \
               or self.model.poi_grid[tr][tc] is None:
                print(" POI objetivo ya no existe; recalculando")
                self.target, self.route = None, []

        if not self.route or (self.target and self.pos == (self.target[1], self.target[0])):
            if self.carry:
                self.plan_to_exit(matriz_pared, access_coords)
            else:
                self.plan_to_nearest_poi(matriz_pared, matriz_POI)

        # Ejecutar movimiento/acciones
        self.follow_route()

        # Interacciones con casilla actual
        row, col = self.pos[1], self.pos[0]
        poi = self.model.poi_grid[row][col]
        if poi and not poi.revealed:
            poi.reveal()
            if poi.is_victim() and not self.carry:
                self.carry = True
                print("Rana cargó una víctima")
                self.model.poi_grid[row][col] = None
                self.plan_to_exit(matriz_pared, access_coords)
            elif poi.is_false():
                print("Rana encontró una falsa alarma")
                self.model.poi_grid[row][col] = None
                self.model.place_new_poi()
                self.target, self.route = None, []

        if self.carry and (row, col) in access_coords:
            self.carry = False
            self.model.rescued += 1
            print("Víctima entregada en salida")
            self.model.place_new_poi()
            self.target, self.route = None, []

        # recarga AP al final
        self.sum_AP()


# ------------------------------
# Modelo
# ------------------------------
class RanaModel(Model):
    def __init__(self, width=8, height=6, agents=6, strategy="astar"):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)

        self.total_damage = 0
        self.max_damage = 24
        self.rescued = 0
        self.lost = 0
        self.rescue_goal = 7
        self.lose_limit = 4
        self.collapsed = False
        self.win_counter = 0
        self.lose_counter = 0

        self.turn_counter = 0   #  contador de turnos
        self.agents_count = agents  #  número de agentes

        self.datacollector = DataCollector(
            model_reporters={
                "Grid": get_grid,
                "Step": lambda m: m.schedule.steps,
                "Rescued": lambda m: m.rescued,
                "Lost": lambda m: m.lost,
                "TotalDamage": lambda m: m.total_damage,
                "Collapsed": lambda m: m.collapsed,
                "WinCount": lambda m: m.win_counter,
                "LoseCount": lambda m: m.lose_counter
            }
        )

        # Crear agentes en los Access Points
        for (row, col) in access_coords:
            agent = RanaAgent(self)
            if 0 <= row < height and 0 <= col < width:
                self.grid.place_agent(agent, (col, row))
                self.schedule.add(agent)

        self.fire_grid = matriz_fire
        self.poi_grid = matriz_POI
        print("Agentes creados:", len(self.schedule.agents))


    def place_new_poi(self):
        # buscar celdas vacías sin fuego ni otro POI
        free = []
        for r in range(len(self.poi_grid)):
            for c in range(len(self.poi_grid[0])):
                if self.poi_grid[r][c] is None and self.fire_grid[r][c].state == Fire.EMPTY:
                    free.append((r, c))
        if not free:
            return
        r, c = random.choice(free)
        # 2/3 víctimas, 1/3 falsa alarma (según reglas simplificadas)
        role = POI.VICTIM if random.random() < 0.66 else POI.FALSE
        self.poi_grid[r][c] = POI(r, c, role)

    def step(self):
        if self.collapsed:
            return

        # turno de un agente
        self.schedule.step()
        self.turn_counter += 1

        # cuando todos los agentes ya actuaron → avanza el fuego
        if self.turn_counter % self.agents_count == 0:
            self.advance_fire()

        self.datacollector.collect(self)

        # Revisar colapso
        if self.total_damage >= self.max_damage and not self.collapsed:
            print("DERROTA: edificio colapsado")
            self.lose_counter += 1
            self.collapsed = True
            return

        if self.rescued >= self.rescue_goal:
            print("VICTORIA: suficientes víctimas rescatadas")
            self.win_counter += 1
            return

        # Explosiones
        for row in self.fire_grid:
            for cell in row:
                if cell.to_explode:
                    cell.propagate_explosion(self.fire_grid, matriz_pared, self)

        # Flashover
        for row in self.fire_grid:
            for cell in row:
                if cell.state == Fire.FIRE:
                    cell.set_smoke(self.fire_grid, matriz_pared)

        # Víctimas perdidas por fuego
        for i in range(len(self.fire_grid)):
            for j in range(len(self.fire_grid[0])):
                poi = self.poi_grid[i][j]
                if self.fire_grid[i][j].state == Fire.FIRE and poi and poi.role == POI.VICTIM:
                    self.lost += 1
                    self.poi_grid[i][j] = None
                    if self.lost >= self.lose_limit:
                        print("DERROTA: demasiadas víctimas perdidas")
                        self.lose_counter += 1
                        return

        #  Mantener siempre al menos 3 POIs activos
        active_pois = sum(1 for row in self.poi_grid for poi in row if poi is not None)
        while active_pois < 3:
            self.place_new_poi()
            active_pois += 1

    def advance_fire(self):
        r = random.randint(0, len(self.fire_grid)-1)
        c = random.randint(0, len(self.fire_grid[0])-1)
        target = self.fire_grid[r][c]

        if target.state == Fire.EMPTY:
            target.state = Fire.SMOKE
        elif target.state == Fire.SMOKE:
            target.state = Fire.FIRE
        elif target.state == Fire.FIRE:
            target.to_explode = True




def get_grid(model):
    """Convierte el grid de Mesa en una matriz numpy para análisis o visualización"""
    grid = np.zeros((model.grid.height, model.grid.width))
    for (content, (x, y)) in model.grid.coord_iter():
        if len(content) > 0:
            grid[y][x] = 1   # usar y como fila, x como columna
    return grid



In [214]:
total_rescued, total_lost, total_wins, total_losses = 0, 0, 0, 0
N = 100

for _ in range(N):
    model = RanaModel()
    for i in range(1000):
        model.step()
        if model.collapsed or model.rescued >= model.rescue_goal or model.lost >= model.lose_limit:
            break
    total_rescued += model.rescued
    total_lost += model.lost
    total_wins += model.win_counter
    total_losses += model.lose_counter


Agentes creados: 6
 Rana irá a puerta cerrada en (4, 3) para abrir/romper
Rana movida a (4, 3)
 Rana no encontró ruta hacia ningún POI ni puerta cerrada alcanzable
Rana planifica hacia POI (0, 0) con ruta [(0, 0)]
Rana movida a (0, 0)
Rana encontró una falsa alarma
 Rana irá a puerta cerrada en (3, 1) para abrir/romper
Rana movida a (3, 1)
 Rana irá a puerta cerrada en (3, 4) para abrir/romper
Rana movida a (3, 4)
 Rana no encontró ruta hacia ningún POI ni puerta cerrada alcanzable
Rana cargó una víctima
 Rana no encontró salida accesible
Víctima entregada en salida
 Rana no encontró ruta hacia ningún POI ni puerta cerrada alcanzable
Rana planifica hacia POI (0, 1) con ruta [(0, 1)]
Rana movida a (0, 1)
Rana encontró una falsa alarma
 Rana no encontró ruta hacia ningún POI ni puerta cerrada alcanzable
 Rana irá a puerta cerrada en (4, 2) para abrir/romper
Rana movida a (4, 2)
 Rana no encontró ruta hacia ningún POI ni puerta cerrada alcanzable
Rana planifica hacia POI (3, 0) con ruta [

/tmp/ipython-input-1116713352.py:208: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in a future version. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


Se han truncado las últimas 5000 líneas del flujo de salida.
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
Daño en pared por fuego en: (2, 1) lado 1
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
DERROTA: edificio colapsado
Agentes creados: 6
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahora mismo
 No hay POIs activos ahor

In [215]:
print("Promedio rescates:", total_rescued/N)
print("Promedio pérdidas:", total_lost/N)
print("Porcentaje victorias:", (total_wins/N)*100)
print("Porcentaje derrotas:", (total_losses/N)*100)

Promedio rescates: 2.7
Promedio pérdidas: 0.15
Porcentaje victorias: 37.0
Porcentaje derrotas: 63.0


In [ ]:
#Conexion local a C#
#Código creado cguiado mediante IA

HOST = "127.0.0.1"
PORT = 65432

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen()
print(f"Servidor escuchando en {HOST}:{PORT}...")

conn, addr = server.accept()
print(f"Cliente conectado desde {addr}")

ITERATIONS = 30
model = RanaModel()


try:
    for i in range(ITERATIONS):
        model.step()
        grid = get_grid(model).tolist()   #convertir numpy → lista

        data = {
            #Funciones
            "step": i,
            "grid": grid,
            #Importar agente
            "agent_pos": ([agent.pos for agent in model.schedule.agents]),
            "agent_ap": ([agent.AP for agent in model.schedule.agents]),
            "agent_carry": ([agent.carry for agent in model.schedule.agents]),
            #Importar fuego
            "fires": ([[cell.row, cell.col, cell.state] for row in model.fire_grid for cell in row if cell.state != Fire.EMPTY]),
            "fire_grid": ([[cell.row, cell.col, cell.state] for row in model.fire_grid for cell in row]),
            #Importar POI
            "pois": ([[poi.row, poi.col, poi.role, poi.revealed] for row in model.poi_grid for poi in row if poi is not None]),
            #Importar paredes
            "wall_grid": ([[cell.get_positions() for cell in row] for row in model.wall_grid]),
            "wall_damages": ([[cell.get_damages() for cell in row] for row in model.wall_grid]),
            #Importar puertas
            "door_list": ([[door.cell1, door.cell2, door.state] for door in door_list]),
            #Modelo
            "model.collapsed": model.collapsed,
            "total_damage": model.total_damage,
            "rescued": model.rescued,
            "lost": model.lost,
            "max_damage": model.max_damage,
        }


        message = json.dumps(data)
        conn.sendall((message + "\n").encode("utf-8"))
        print(f"Enviado paso {i}")

finally:
    conn.close()
    server.close()